In [54]:
# Standard Imports
import numpy as np 
import pandas as pd
from math import log
from sklearn.model_selection import train_test_split

### Discrete Feature Discrete Output Decision Tree for Classification
Note - Please rename output variable to be 'Class' in your dataset for this implementation to work

In [55]:
def entropy(target):
    freq = {}
    for elem in target:
        if elem not in freq:
            freq[elem]=1/len(target)
        else: 
            freq[elem]+=1/len(target)
    _sum = [-freq[i]*np.log2(freq[i]) for i in freq]
    return sum(_sum)

def info_gain(data,attributes):
    entropy_dataset = entropy(data['Class'])
    entr_val = {}
    infg = {}
    for attr in attributes:
        freq = {}
        for i in data[attr]:
            if i not in freq:
                freq[i]=1/len(data[attr])
            else:
                freq[i]+=1/len(data[attr])
        keys = list(freq.keys())
        entr = {}
        for i in keys:
            temp = len(data[data[attr]==i])
            entr[i] = temp/len(data)*entropy(data[data[attr]==i]['Class'])
        entr_val[attr] = sum(list(entr.values()))
        infg[attr] = entropy_dataset - entr_val[attr]
    return max(infg, key = infg.get)

def dtree(dataset,data,attributes,parent):
    attribute = [i for i in attributes]
    if len(data['Class'].unique())<=1:
        return (list(data['Class'].unique())[0])
    elif len(data)==0:
        return np.unique(data['Class'])[np.argmax(np.unique(data['Class'],return_counts=True)[1])]
    elif len(attribute)==0:
        return parent
    else:
        parent = np.unique(data['Class'])[np.argmax(np.unique(data['Class'],return_counts=True)[1])]
        
    best = info_gain(data,attribute)
    tree = {best:{}}
    attribute.remove(best)
    for ndata in data[best].unique():
        new_data = data[data[best]==ndata]
        subtree = dtree(dataset,new_data,attribute,parent)
        tree[best][ndata]=subtree
    return tree  

### Continuous Features Discrete Output Decision Tree for Classification
The code was majorly referred from this link here. 

In [348]:
## Iris Dataset Classification
def gini(splits,classes):
    total_rows = 0
    final = 0
    for i in range(len(splits)):
        total_rows+=len(splits[i])
    final = 0
    for split in splits:
        if len(split)==0:
            continue
        gscore = 0
        for cls in classes:
            p = [row[-1] for row in split].count(cls)/len(split)
            gscore+=p*p
        final+= (1-gscore)*(len(split)/total_rows)
    return final
## This needs to be done very carefully
## Binary Splits
def divide_data(dataset,feature,threshold):
    new_data1 = []
    new_data2 = []
    for elem in dataset:
        if elem[feature]<threshold:
            new_data1.append(elem)
        else:
            new_data2.append(elem)
    return new_data1, new_data2

def best_split(dataset,classes):
    best_so_far = 1000
    best_splits = None
    feature = 1000
    value_split = 1000
    for i in range(len(dataset[0])-1):
        for elem in dataset:
            splits = divide_data(dataset,i,elem[i])
            gscore = gini(splits,classes)
            if gscore<best_so_far:
                best_so_far = gscore
                best_splits = splits
                feature = i
                value_split = elem[i]
    return {'split':best_splits,'Feature':feature,'Value':value_split}
### Metrics Done _ Build Tree now .....
def leaf_node(split):
    cls = [elem[-1] for elem in split]
    # Take Majority Vote
    count = {}
    for i in cls:
        if i not in count:
            count[i]=1
        else:
            count[i]+=1
    return max(count, key = count.get)

def partition(node, maxdepth, minsize, depth):
    ## Each partition from above gives a node in essence.
    lchild, rchild = node['split']
    if not lchild or not rchild:
        node['left']= leaf_node(lchild+rchild)
        node['right'] = leaf_node(lchild+rchild)
        return
    if depth>maxdepth:
        node['left'] = leaf_node(lchild)
        node['right'] = leaf_node(rchild)
        return
    if len(lchild)<=minsize:
        node['left'] = leaf_node(lchild)
    else:
        node['left'] = best_split(lchild,cls)
        partition(node['left'],maxdepth,minsize, depth+1)
    if len(rchild)<=minsize:
        node['right'] = leaf_node(rchild)
    else:
        node['right'] = best_split(rchild,cls)
        partition(node['right'],maxdepth,minsize,depth+1)
        
def print_tree(node,depth):
    if type(node)==dict:
        print('%s[X%d < %.3f]' % ((depth*' ', (node['Feature']+1), node['Value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('Depth - %s[%s]' % ((depth*' ', node)),depth)


def tree_iris(dataset, maxdepth, minsize):
    root = best_split(dataset,cls)
    partition(root,maxdepth,minsize,1)
    ## Printing the made Decision Tree
    return root

def predict(node, row):
	if row[node['Feature']] < node['Value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']


"""def predict(node, elem):
#     print(node,elem)
    if elem[node['Feature']]<node['Value']:
#         print(node['Value'],elem[node['Feature']])
        
        if type(node['left'])==dict:
            predict(node['left'],elem)
        else:
#             print('---printining left---',node['left'])
            return node['left']
    else:
#         print('Greater than')
        if type((node['right']))==dict:
            predict(node['right'],elem)
        else:
#             print('---printining right ---',node['right'])
            return node['right']"""
def class_aggregate(dataset):
    temp = []
    for i in range(len(dataset)):
        temp.append(dataset[i][-1])
    temp = np.unique(np.array(temp))
    return temp
cls = class_aggregate(np.array(df))
cls = np.uint8(cls)
def val_replace(cls,dataset):
    for i in range(len(cls)):
        dataset = dataset.replace(cls[i],i)
    return dataset
df = val_replace(cls,df)

def accurate(test,tree):
    temp = 0
    for i in test:
        val = predict(tree,i)
        if i[-1]==val:
            temp+=1
    return temp/len(test)*100

In [ ]:
accuracy = {}

In [406]:
for i in range(1,20):
    s =0
    for j in range(100):
        train, test = train_test_split(np.array(df),test_size = .30,random_state = j)
        tree = tree_iris(train,i,0)
        s+=(accurate(test,tree))
        

91.11111111111111
88.88888888888889
93.33333333333333
93.33333333333333
95.55555555555556
91.11111111111111
93.33333333333333
95.55555555555556
91.11111111111111
91.11111111111111
97.77777777777777
95.55555555555556
93.33333333333333
88.88888888888889
88.88888888888889
88.88888888888889
95.55555555555556
97.77777777777777
95.55555555555556


In [ ]:
def kfold(k,)

### Decision Tree Regression